In [ ]:
# pip install schedule
# pip install -- upgrade schedule

In [ ]:
import schedule  
import time  
import pandas as pd
import pyodbc 

def job():  
    server = 'DESKTOP-KLPI1BO\\SQLEXPRESS' 
    database = 'car_shop' 
    conexaoDB = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                               f'SERVER={server};'
                               f'DATABASE={database};'
                               'Trusted_Connection=yes;')

    cursor = conexaoDB.cursor()   # criando cursor de comando 

    dados = pd.read_excel(r"C:\Users\Luiz Krawiec\OneDrive\Documentos\portfolio\automação, implantação e EDA\excel\tbCustomers.xlsx")
    str(dados.columns).replace("'", "")

    # limpa banco de dados antes da carga 
    cursor.execute('TRUNCATE TABLE [tbCustomers3]')  
    conexaoDB.commit()

    # faz carga no banco de dados
    for index, linha in dados.iterrows():
        # Truncate customer_id if it exceeds 50 characters
        customer_id = str(linha.customer_id)[:50]
        cursor.execute("INSERT INTO [tbCustomers3] (customer_id, cpf, first_name, last_name, state, city, birth_date, income, score, professional_status, email, mobile) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                       (customer_id, str(linha.cpf), linha.first_name, linha.last_name, linha.state, linha.city, str(linha.birth_date), int(linha.income), int(linha.score), linha.professional_status, linha.email, str(linha.mobile)))
        
    conexaoDB.commit()   
    cursor.close() 
    conexaoDB.close() 
    
schedule.every().day.at("12:42").do(job)   # escolher a frequencia 
while True: # loop contínuo
    schedule.run_pending()  # Executa tarefas agendadas que estão prontas para serem executadas
    time.sleep(1)  # Pausa por 1 segundo antes de verificar novamente as tarefas agendadas
